<a href="https://colab.research.google.com/github/techquila192/Gamma-Hadron-Classification/blob/main/Untitled2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import RandomOverSampler
from sklearn.metrics import accuracy_score

In [35]:
cols=['fLength', 'fWidth', 'fSize', 'fConC', 'fConc1', 'fAsym', 'fM3Long', 'fM3Trans', 'fAlpha', 'fDist', 'class']
df = pd.read_csv("/content/sample_data/magic04.data",names=cols)
df_train,df_test,df_valid = np.split(df.sample(frac=1),[int(0.6*len(df)),int(0.8*len(df))])
df_train.head(10)

,fLength,fWidth,fSize,fConC,fConc1,fAsym,fM3Long,fM3Trans,fAlpha,fDist,class
2153,32.3022,14.8135,2.7832,0.4382,0.2595,40.9266,23.3847,12.3260,10.8360,131.5200,g
14647,25.8010,21.5157,2.6187,0.3049,0.1519,16.8513,13.8349,-14.1239,29.4336,142.0574,h
6903,33.4992,12.0120,2.6248,0.4413,0.2859,38.7827,20.6561,11.1602,16.2492,197.9290,g
17975,123.1790,24.6852,3.3411,0.2585,0.1411,-75.3966,69.1339,18.4605,8.9870,343.2420,h
17279,20.7522,8.9492,2.3874,0.6434,0.3586,24.0649,10.5927,1.8768,36.3060,126.3930,h
13234,77.1446,38.0002,3.6371,0.1518,0.0956,-33.1911,70.3046,29.5218,31.7121,285.0727,h
8069,30.9768,12.5732,2.6972,0.4458,0.2620,26.9816,26.4698,9.7728,10.8290,182.3420,g
17167,96.2503,44.8200,3.3839,0.0817,0.0292,-74.6218,-95.1478,43.5440,17.2484,218.1069,h
5781,25.6507,15.6521,2.6527,0.4672,0.2570,22.8993,22.1003,-5.9987,0.6530,153.3920,g
8047,21.8049,9.0505,2.3054,0.6287,0.3441,4.5959,-16.0107,-10.7749,60.7750,58.9665,g


# Data Scaling and Resampling

In [36]:
def scale_data(df):
    df["class"]=(df["class"]=="g").astype(int)
    df=df.fillna(df.median())
    scaler=StandardScaler()
    x_scale=df[cols[:-1]].values
    y=df[cols[-1]].values
    x_scaled=scaler.fit_transform(x_scale)
    x=x_scaled
    ros=RandomOverSampler()
    x,y=ros.fit_resample(x,y)
    data=np.hstack((x,np.reshape(y,(-1,1))))
    return x,y,data

In [37]:
x_train,y_train,train=scale_data(df_train)
x_test,y_test,test=scale_data(df_test)
x_valid, y_valid, valid=scale_data(df_valid)

# Using Naive Bayes (Gaussian)

In [40]:
from sklearn.naive_bayes import GaussianNB
model=GaussianNB()
NB_model=model.fit(x_train,y_train)

In [43]:
y_pred=NB_model.predict(x_valid)
print(accuracy_score(y_valid,y_pred))

0.6549153908138597


# Training using Support Vector Machine(SVM)

In [38]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report
model=SVC(kernel="rbf")
SVM_model=model.fit(x_train,y_train)

In [39]:
y_pred=SVM_model.predict(x_valid)
print(accuracy_score(y_valid,y_pred))

0.838436744560838


# Using KNN
Evaluating using k values from 1 to 20, it was found that the highest accuracy is achieved using a k value of 17

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
model=KNeighborsClassifier(n_neighbors=17)
KNN_model=model.fit(x_train,y_train)
y_pred=KNN_model.predict(x_valid)
print(accuracy_score(y_valid,y_pred))


0.8118630248675092


# Final Model
Better results are achieved using SVM

In [ ]:
print(accuracy_score(y_test,SVM_model.predict(x_test)))

0.850996015936255
